# **Practical Assignment 2: Real-time Object Detection System Development and Solution Study**

In this Practical Assignment, you will be performing fine-tuning of an object detection model, which may consume too much resources for Google Colab (free version) to handle. Thus we will try to run the code on our personal laptops, which also makes it easier to connect to our laptop webcam.

**Note**: The base environment provided by Anaconda may not have the correct library versions installed. It's best to start from a new "environment", and you should be opening up this Notebook from inside the new environment we will create together in class.

# 1. Requirement
This is a capstone assignment, in which you will learn practical object detection technology and solution. The assignment will guide you to develop a real-time object detection system with updated YOLO framework, and you will engage in end-to-end practice, encompassing the entire pipeline from image labelling, model finetuning to webcam deployment. In details, the assignment is comprised of three parts:
* Finetune a basic YOLO model for wall crack detection. 
* Retrain the model for live video detection, object tracking and counting.  
* Explore the solutions for diverse object detection applications.

Referring to '**practical_assignment_2.docx**' for the detailed instructions, write down experiment steps, results, answers and literature study into '**practical_assignment_2.docx**', and expand it into your final assignment report.

# 2. Fine-tune YOLO11 model for crack detection

## 1. Set up development environment

### Install ultralytics library

And run some system analysis :)

In [ ]:
# The first time you run the new environment, need to install ultralytics packages and dependencies
%pip install numpy matplotlib
%pip install ultralytics

# Load the libraries to make sure everything's fine
import matplotlib.pyplot as plt
import ultralytics

# So that our matplotlib plots show up in the notebook itself
%matplotlib inline

# Check software and hardware
ultralytics.checks()

### Run inference on a test image

The detection results will open in a new window

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")  # initialize model
results = model(["https://ultralytics.com/images/bus.jpg"])  # perform inference
results[0].show()  # display results for the first image

### Set working directories / file paths

Please change the following file paths to match where you saved the data

In [6]:
import os

path_PA2 = r'C:\Users\xiaoming\PA2'                       # (Windows) This should be the directory where your PA2 files are all stored
                                                                       # On Mac/Linux, file paths will look like '/Users/S11009880/...' or '/home/S1100...'
path_yaml = os.path.join(path_PA2, r'datasets\crack400.yaml')          # Config file for YOLO model training
saved_model = os.path.join(path_PA2, 'best.pt')                        # Model weights for our best-performing trained model
path_test = os.path.join(path_PA2, r'datasets\crack400\images\test') # Test dataset images


## 2. Prepare cracks dataset

- Download and unzip `datasets.zip`, which contains images of wall/pavement cracks
  - Currently, all images are in the `raw` folder
  - The directory structure for model training and evaluation has been set up for you

## 3. Label / annotate images

We will annotate our images using the LabelImg program
- Setup:
  - Download from Teams and unzip `labelImg.zip`
  - Prepare for dataset annotation by changing predefined_classes.txt to only have one class: crack

- Annotate the `train`, `val`, and `test` images

- Move images (`.jpg`) and annotations (`'.txt`) into respective folders

- Move `test` and `val` background images

## 4. Create dataset config file

* Change the `path` to where your `crack400` folder is
  * You should not need to change the paths for the train/val/test directories
  * Ensure that the class IDs and names are correct

## 5. Train and evaluate model

Here, we will load a pre-trained model and train it on our cracks dataset.

You are recommended to use `'yolo11n.pt'`. A suitable (older) alternative model is `'yolov8n.pt'`.

When running the following cell, the last line will start the training process, and displays the following outputs:
1. The full set of training parameters used (inlcuding the defaults)
2. The architecture of the YOLO model (much like the PyTorch model summaries we saw in CNNs for classification)
3. Some status information for the loading of train and val datasets
4. Progress bars that will be updated as model training goes on for each epoch
   - After each epoch, the model is evaluated against the Val dataset, giving the **P**recision, **R**ecall, **mAP50**, and **mAP50-95**.
5. Evaluation reaults of the best model so far

**Note**: The last lines in the output will indicate where the trained weights are saved.

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO model
model1 = YOLO('yolo11n.pt')

# Train the model on crack dataset for 5 epochs
num_epochs = 5  # Number of epochs
lr0 = 0.01      # What do lr0 and lrf represent? See documentation at https://docs.ultralytics.com/modes/train/#train-settings
lrf = 0.01
batch = 8       # Batch size

results = model1.train(data=path_yaml, epochs=num_epochs, lr0=0.01, lrf=0.01, batch=8)

After training for a few epochs, we will load the best model obtained and evaluate it against the **Test** dataset. 

**Note**:
- The last line in the output above should read: `Results saved to ...\runs\detect\trainX`, where `X` is some number
  - Set `traindir` to this directory to this directory!
  - Each run will create a different folder for weights to be stored.

In [ ]:
# Load best YOLO model
traindir = r'C:\Users\xiaoming\PA2\runs\detect\train2'
weightsfile = os.path.join(traindir, r'weights\best.pt')
model2 = YOLO(weightsfile)

# Evaluate the model's performance on the test set
results = model2.val(data=path_yaml, split='test')

### **[Task 1]** Study the dataset and output information, answer the questions below, and write into your report. <br>

 * How many images have you labelled for training, validation and test respectively?
 * What is original image size (H x W)?
 * What is input image size (H x W) required by the chosen YOLO network?
 * What is your model (best.pt) file size?
 * What is your model optimizer and learning rate (lr)?
 * How many layers and parameters does your model have?
 * What is number of classes (attribute `nc`) in pretrained YOLO model (`model1`)? And what is number of classes (`nc`) in your best.pt model (`model2`)?
 * What is your model (best.pt) mAP50 and mAP50-95 on validation set?
 * What is your model (best.pt) mAP50 and mAP50-95 on test set?
 * Among accuracy, recall and precision, which metric is more important for the building quality assurance? 


## 6. Finetune the model

Reload the pre-trained model, and try training + evaluating the trained model using different training parameters.

In [ ]:
# Following the Task 2 below, retrain and evaluate the model
# Referring to https://docs.ultralytics.com/modes/train/#usage-examples, adjust training arguments (hyperparameters)

# Load a pretrained YOLO model
model1 = YOLO('yolo11n.pt')
# Retrain the model 
num_epochs = 10
lr0 = 0.01     
lrf = 0.01
batch = 8      

results = model1.train(data=path_yaml, epochs=10, lr0=0.01, lrf=0.01, batch=8)

Reload the best model weights, and evaluate on Test dataset

In [ ]:
# Evaluate new model's performance on the test set
traindir = r'C:\Users\xiaoming\PA2\runs\detect\train10'
weightsfile = os.path.join(traindir, r'weights\best.pt')
model2 = YOLO(weightsfile)
results = model2.val(data=path_yaml, split='test')

Plot and save the detections annotated on test images. The output will indicate where to find the annotated test images.

In [ ]:
# Test the model with test images:
results = model2.predict(path_test, save=True)

### **[Task 2]** Write down experiment steps, settings, results and conclusions to the report. <br>
 * Adjust epochs with different values like 10, 20, 30, 50 and 100, retrain the model, study how epoch value impacts model performance (mAP50 and mAP50-95) on test set.
 * Adjust learning rate (lr0 and lrf) with different values like 0.01, 0.02, 0.03, 0.001, 0.002, 0.003, retrain the model, study how learning rate impacts model performance (mAP50 and mAP50-95) on test set.
 * Adjust batch with different size like 8, 16, 32, retrain the model, study how batch size impacts model performance (mAP50 and mAP50-95) on test set.
 * Check ‘confusion_matrix.png’ in ‘runs/detect/val/’, write down metrics (TP, FP, TN, FN) in confusion matrix.
 * Copy all the images from ‘train_background’ folder to ‘train’ folder.
   * If you had already included the train_background pictures in training, try deleting these backgroud images (the file names all start with `bg`) and then rerun the training and evaluation.
 * Retrain the model and validate the model with test set.
 * Check ‘confusion_matrix.png’ in ‘runs/detect/val/’, study how the metrics (TP, FP, TN, FN) in confusion matrix are impacted after adding background images to training set.
 * Test the model through ‘model2.predict()’ command given above, find prediction results in ‘runs/detect/prediction#’ folder, and check whether the detection is accurate.

**Note from Zhen Wah**: Part 2 will be in the next notebook!